Analyzing some 911 call data from [Kaggle](https://www.kaggle.com/mchirico/montcoalert). The data contains the following fields:

* lat : String variable, Latitude
* lng: String variable, Longitude
* desc: String variable, Description of the Emergency Call
* zip: String variable, Zipcode
* title: String variable, Title
* timeStamp: String variable, YYYY-MM-DD HH:MM:SS
* twp: String variable, Township
* addr: String variable, Address
* e: String variable, Dummy variable (always 1)

## Data and Setup

** Let's import essential libraries: numpy, pandas, seaborn and matplotlib **

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

** Put matplotlib inline to make the plot show up in here **

In [ ]:
import seaborn as sns
%matplotlib inline

** Read in the csv file as a dataframe called df **

**(將資料用padnas中的read_csv()讀進來)**

In [ ]:
df = pd.read_csv('../input/911.csv')

** Check the info() of the df **

In [ ]:
df.info()

** As we could see from the above, we got 266543 rows of data with 9 different arrtibutes**

** Check the head of df **

In [ ]:
df.head()

** We could see that there is some NA value inside the dataset, and it should be careful when prepara the data **

## Basic Questions

** What are the top 5 zipcodes for 911 calls? **

** (針對df['zip']內裡面的數值先使用value_counts()整理出來，在使用nlargest()找出前五名)**

In [ ]:
df['zip'].value_counts().nlargest(5)

In [ ]:
sum(df['zip'].value_counts().nlargest(5))/sum(df['zip'].value_counts()) * 100

** What are the top 5 townships (twp) for 911 calls? **

**(針對df['twp']內裡面的數值先使用value_counts()整理出來，在使用nlargest()找出前五名)**

In [ ]:
df['twp'].value_counts().nlargest(5)

In [ ]:
sum(df['twp'].value_counts().nlargest(5))/sum(df['twp'].value_counts()) * 100

** Take a look at the 'title' column, how many unique title codes are there? **

**(在資料中查找'titile'的資料，使用unique()函數先取出結果(沒想到型態是ndarray)，在使用size看有多少筆)**

In [ ]:
df['title'].unique().size
#len(df['title'].unique()) # Another way to get the result

### Section Summary
**Indeed, the top 5th zip & township weight around 30% of total call in this region. Maybe we could take a look what kind of calls are made here? **

## Creating new features

** In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.** 

**For example, if the title column value is EMS: BACK PAINS/INJURY , the Reason column value would be EMS. **

**We need to spilt the title column by using split() by leveraging lambda. And we got the first result ( index 0 ) as what we expected**

In [ ]:
reason = df['title'].apply(lambda x: x.split(':')[0])
df['Reason'] = reason

** What is the most common Reason for a 911 call based off of this new column? **

In [ ]:
df['Reason'].value_counts()

** Now use seaborn to create a countplot of 911 calls by Reason. **

In [ ]:
sns.countplot(df['Reason'])
#sns.countplot(x='Reason', data= df) #Another way to plot it

___
**Section Objective**

** In the following sections, we will start to investigate the call reasons with hour, month and day of week. Let's start from day of week and month**

____

** Now let us begin to focus on time information. What is the data type of the objects in the timeStamp column? **

**In order to analyze the frequency of the call by hour, day, month.....etc, let's convert the timeStamp column into specific data**

In [ ]:
type(df['timeStamp'][0])

** The timestamps type are still strings. Use [pd.to_datetime](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) to convert the column from strings to DateTime objects. **

In [ ]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])

** Some reference to locate the data by index**

    time = df['timeStamp'].iloc[0]
    time.hour

In [ ]:
hour = df['timeStamp'].apply(lambda x: x.hour)
month = df['timeStamp'].apply(lambda x: x.month)
day =  df['timeStamp'].apply(lambda x: x.weekday())
#day = df['timeStamp'].apply(lambda x: x.dayofweek) #Another way tp get the weekday

In [ ]:
# As we will got the weekday in number from .weekday(), we could create a dictionary with key-value pair of its name.
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
week_day = day.map(dmap)

In [ ]:
df['Hour'] = hour
df['Month'] = month
df['Day of Week'] = week_day

** Now use seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column. **

In [ ]:
sns.countplot(x = df['Day of Week'],hue=df['Reason']).legend(loc='center left', bbox_to_anchor=(1, 0.5))


**Now do the same for Month:**

In [ ]:
sns.countplot(df['Month'],hue=df['Reason']).legend(loc='center left', bbox_to_anchor=(1, 0.5))

**There are some months are missed in this dataset**

_____


** Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation. Use the head() method on this returned DataFrame. **

In [ ]:
byMonth = df.groupby('Month').count()
byMonth
#As we notice before, there are some NA values inside which caused the count() number difference

** Now create a simple plot off of the dataframe indicating the count of calls per month. **

In [ ]:
byMonth['twp'].plot()

** Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column. **

In [ ]:
byMonth['index'] = byMonth.index
sns.lmplot(x='index', y='twp',data=byMonth)
#sns.lmplot(x='Month', y='twp',data = byMonth.reset_index()) #Another way to plot it

**Create a new column called 'Date' that contains the date from the timeStamp column. You'll need to use apply along with the .date() method. ** 

In [ ]:
date = df['timeStamp'].apply(lambda x: x.date())
df['Date']= date
#注意TimeStamp object的特性

** Now groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.**

In [ ]:
byDate = df.groupby('Date').count()
byDate['twp'].plot()
plt.tight_layout()

** Now recreate this plot but create 3 separate plots with each plot representing a Reason for the 911 call**

In [ ]:
df[df['Reason']=='Traffic'].groupby('Date')['twp'].count().plot()
plt.tight_layout()

In [ ]:
df[df['Reason']=='Fire'].groupby('Date')['twp'].count().plot()
plt.tight_layout()

In [ ]:
df[df['Reason']=='EMS'].groupby('Date')['twp'].count().plot()
plt.tight_layout()

____
**Section Summary**

** We could see from both histogram and count plot that calls for the reason EMS are the highest among the time period**
____
** Now let's move on to creating  heatmaps with seaborn and our data. We'll first need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week. There are lots of ways to do this, but I would recommend trying to combine groupby with an [unstack](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) method. Reference the solutions if you get stuck on this!**

In [ ]:
day_hour = df.groupby(['Day of Week','Hour']).count()['Reason'].unstack()
day_hour

** Now create a HeatMap using this new DataFrame. **

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(day_hour, cmap="viridis")

** If we see from the hour's point of view, the call count during midnight( 0~6 ) is low which is reasonable because people are sleeping. And high call counts located in 15~18 this period during weekdays. **

** Now create a clustermap using this DataFrame. **

In [ ]:
sns.clustermap(day_hour,cmap='viridis')

** Now repeat these same plots and operations, for a DataFrame that shows the Month as the column. **

In [ ]:
month_hour = df.groupby(['Day of Week','Month']).count()['Reason'].unstack()
month_hour

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(month_hour,cmap='viridis')

In [ ]:
sns.clustermap(month_hour,cmap='viridis')

## Summary
This is a basic data analysis report for 911 calls. And we could got some preliminary conclusions from the investigation.
1. EMS are the most 911 call reasons
2. Higher 911 call happens during the weekday's afternoon( 15~18 )
3. Compared to weekday's call frequency and count, weekend are relative quiet

## Follow-Up Investigate
Some data we are not leverage, ex: latitude, longitude and address. We could further investigate which location made what kind of call the most, and dig out what kind of people living around here as reference.